#COPY INTO & MERGE commands


###COPY INTO command
- incrementally loads data into delta table from cloud storage.
- supports schema evolution 
- supports wide range of file format (CSV,JSON,parquet,delta)
- Alternative to autoloader for batch ingestion

###Create Table to copy data into

In [0]:
%sql
CREATE TABLE IF NOT EXISTS demo.delta_lake.raw_stock_prices;

###Incrementally load new files into the table

In [0]:
%sql
SHOW SCHEMAS

In [0]:
DELETE FROM demo.delta_lake.raw_stock_prices;

COPY INTO demo.delta_lake.raw_stock_prices
FROM 'abfss://demo@deacourseex.dfs.core.windows.net/landing/stock_prices'
FILEFORMAT = JSON
FORMAT_OPTIONS('inferSchema' = 'true')
COPY_OPTIONS('mergeSchema' = 'true');

In [0]:
SELECT * FROM demo.delta_lake.raw_stock_prices

###MERGE statement
- used for upserts (insert/update/delete operations in single statement)
- Allows merging new data into target table based on matching condition

###Create table to merge table into

In [0]:
CREATE TABLE IF NOT EXISTS demo.delta_lake.stock_prices
(
  stock_id STRING,
  price DOUBLE,
  trading_date DATE
);

###Merge source data into target table
- insert new stocks received
- update prices and trading_date if updates recieved
- delete stocks which are de-listed from exchange(status = 'DELISTED')

In [0]:
MERGE INTO demo.delta_lake.stock_prices AS target
USING demo.delta_lake.raw_stock_prices AS source
  ON target.stock_id = source.stock_id
WHEN MATCHED AND source.status = 'ACTIVE' THEN
  UPDATE SET target.price = source.price, target.trading_date = source.trading_date
WHEN MATCHED AND source.status = 'DELISTED' THEN
  DELETE
WHEN NOT MATCHED and source.status = 'ACTIVE' THEN
  INSERT (stock_id, price, trading_date) VALUES (source.stock_id, source.price, source.trading_date);

In [0]:
SELECT * FROM demo.delta_lake.stock_prices;